# Tests

## New API points

### Molecule

* `__init__(hierarchy_schemes=hierarchy_flavor)`
* merge_molecules
* smirks_reaction(?)
* mutate
* perceive_residues

```python
def test_topology_perceive_residues_iterator(self):
    """Test residue iterator perception."""
    expected_residues = {...}
    protein = Molecule.from_file(...)
    protein.perceive_residues()
    # Can we make sure the order is always the same? JW -- Let's not check order in this test, just test for dict equivalence
    # expected residues is a dict of the form {('A',1, 'ALA'):<HierElement with indices 0..10>, ...}
    assert len(expected_residues) == len(protein.residues)
    for residue_tuple, residue_hier_ele in expected_residues:
        assert protein.residues[residue_tuple] == residue_hier_ele
       
    #assert protein.residues == expected_residues
    [atom.name for atom in residues[0].atoms]
    [protein.atoms[at_idx] for at_idx in residues[0].atom_indices]
    ele = HierarchyElement(atom_indices, hierarchy_scheme=None)

```

* perceive_hierarchy
* @classmethod? register_chemical_residue_substructures({residue SMARTS: (residue name, atom names)})
* ??register_typed_residue_substructures({residue networkx graph: ([formal charges], [bond orders])})
   * This could be redundant with the method above
* deregister_residue_substructures
* register_hierarchy_scheme
* deregister_hierarchy_scheme
* hierarchy_schemes

```python
def test_molecule_default_none_hierarchy_schemes(self):
    """Test default and empty hierarchy flavors."""
    offmol = Molecule()
    assert HIER_FLAVOR_DEFAULT == offmol.hierarchy_schemes
    offmol = Molecule(hierarchy_flavor=HIER_FLAVOR_NONE)
    assert {} == offmol.hierarchy_schemes
```

* from_pdb_file
    * _from_pdb_with_atom_and_residue_names(typ_mol)
    * _from_pdb_with_conect(typ_mol) # uses _registered_residue_substructures
* residues (and other iterators) 
* to_openeye (update)
* to_rdkit (update)
* to_dict (update/obliterate with pydantic serialization)
* to_file (update)
* add_conformers (make better API -- let users specify a correct shaped array)


#### Troubleshooting/caveats

In [ ]:
top = Topology.from_molecules([water, protein, ethanol])

top.chains[1].residues[5]

res = top.residues[0]
res.atoms[0].index(reference='topology')
> 3 # The whole-topology index of the atom
 # OR
res.atoms[0].topology_atom_index
> 3 # The whole-topology index of the atom
res.atoms[0].reference_molecule_atom_index == res.atoms[0].index(reference='reference_molecule')
> 0 # The index of the atom in the reference molecule
res.atoms[0].topology_molecule_atom_index  == res.atoms[0].index(reference='topology_molecule')
> 0 # The "local" index of the atom in the topology molecule

ele = top.topology_molecules[5].residues[0]
ele.atoms[0].topology_atom_index
> 200
ele.atoms[0].topology_molecule_atom_index
> 0
ele.atoms[0].index
> 0

ele = top.residues[500]
ele.atoms[0].topology_atom_index
> 200
ele.atoms[0].topology_molecule_atom_index
> 0
ele.atoms[0].index
> 200


type(res.atoms[0])
> TopologyAtom


res = protein.residues[0]
res.atoms[0].index
> 0
type(res.atoms[0])
> Atom

 
class HierarchyElementDefinedWithAtomIndices:
    def __init__(atom_indices, hierarchy_scheme):
        self._topology_molecule_atom_indices = atom_indices
        self._hierarchy_scheme = hierarchy_scheme
        
    def topology_atom_indices(self):
        for top_mol_at_idx in self._topology_molecule_atom_indices:
            yield self.hierarchy_scheme.molecule.local_idx_to_top_idx(top_mol_at_idx)
        
    def identifier(self):
        if self._cached_identifier is None:
            for identifier, element in self.hierarchy_scheme.elements.items():
                if element==self:
                    self._cached_identifier = identifier
                    break
        return self._cached_identifier
    
    def atoms(self) --> Atom or TopologyAtom:
        mol = self.hierarchy_scheme.molecule
        for at_idx in self.atom_indices:
            if isinstance(mol, Molecule):
                yield mol.atoms[at_idx]
            elif isinstance(mol, TopologyMolecule):
                yield mol.topology_atoms[at_idx]

class HierarchyElementDefinedWithAtoms:
    def __init__(atoms, hierarchy_scheme):
        self._atoms = atoms
        self._hierarchy_scheme = hierarchy_scheme
    
    def atom_indices(self):
        for atom in self.atoms:
            if isinstance(atom, Atom)
                yield atom.molecule_atom_index
            elif isinstance(atom, TopologyAtom):
                yield atom.topology_atom_index
                
[atom.name for atom in hierarchy_elemen.atoms]


atom = topology.chains[1].residues[5].atoms[7]
atom.residue
> <HierElement with name 'residues' with id ('A', 10, 'ALA')>
atom.residue.chain
> <HierElement with name 'chains' with id ('A',)>
type(atom)
> AtomView
atom.reference_atom
> Atom in some Molecule
atom
> <AtomView of <AtomView of <AtomView of <Atom in some Moleucle>>>>
atom.index(reference='residue')
> 7

atom2 = topology.atoms[105]
atom2.chain
> AttributeError???
type(atom2)
> TopologyAtom

### Atom

* metadata (dataclass preferred, but dict would work too)



### TypedMolecule

Much like the "normal" stuff, just not being able to return formal charge or bond orders.

* `__init__(**kwargs)`
* add_atom(mass, element, **kwargs)
* add_bond(atom1, atom2, **kwargs)
* add_virtual_site(atoms, n_particles, **kwargs)
* atoms
* bonds
* virtual_sites
* particles
* n_atoms
* n_bonds
* n_virtual_sites
* n_particles
* angles/propers/impropers(?optional?)
* add_conformer
* conformers
* to_file (initially just support PDB, possibly as typ_mol.to_mdtraj().to_pdb() )
* to_networkx()
* from_file (initially just PDB)


### TypedBond

* `__init__(atom1, atom2, molecule, **kwargs)`
* atom1
* atom2
* atom1_index
* atom2_index



### TypedAtom
* `__init__(mass, element, molecule=None, **kwargs)`
* molecule_atom_index
* molecule_particle_index

### TypedVirtualSite
* `__init__(atoms, n_particles, **kwargs)`
* particles

```python
class TypedVirtualSite(BaseModel):
    def __init__(self, atoms, n_particles, **kwargs):
        self.atoms = atoms
        self.n_particles = n_particles
        for kwarg in kwargs:
            self.some_dict[kwarg] = kwargs[kwarg]
            
    def particles(self):
        for i in range(self.n_particles):
            yield TypedVirtualParticle(self, i)
            
    def molecule_virtual_site_index(self):
        return self.atoms[0].molecule.virtual_sites.index(self)
    
```

### TypedVirtualParticle

* `__init__(virtual_site, orientation, **kwargs)`
* molecule_particle_index


### Topology

* `__init__()`

```python
def test_topology_init_no_hierarchy(self):
    """Test a new hierarchy scheme is succesfully registered in the topology."""
    topology = Topology()
    # Empty hier scheme for empty topology or default flavors?
    # This may be more suited for the constructor of the class.
    assert topology.hierarchy_schemes == {}
    
def test_topology_init_with_hierarchy_flavor(self):
    """Test a new hierarchy scheme is succesfully registered in the topology."""
    topology = Topology(hierarchy_schemes=HIER_FLAVOR_DEFAULT)
    # Empty hier scheme for empty topology or default flavors?
    # This may be more suited for the constructor of the class.
    assert len(HIER_FLAVOR_DEFAULT) == len(topology.hierarchy_schemes)
    for hier_scheme in HIER_FLAVOR_DEFAULT:
        assert topology.hierarchy_schemes[hier_scheme.name] == hier_scheme
    
def test_topology_init_with_hierarchy_schemes(self):
    """Test a new hierarchy scheme is succesfully registered in the topology."""
    hier_schemes = [HierarchyScheme(name='residues', 
                                    uniqueness_criteria = ['chain', 'residue_num', 'residue_name']),
                    HierarchyScheme(name='chains',
                                    uniqueness_criteria = ['chain'])
                   ]
    topology = Topology(hierarchy_schemes=hier_schemes)
    assert len(topology.hierarchy_schemes) == 2
    for hier_scheme in hier_schemes:
        assert topology.hierarchy_schemes[hier_scheme.name] == hier_scheme

def test_topology_init_with_incompatible_hierarchy_schemes(self):
    hier_schemes = [HierarchyScheme(name='residues', 
                                    uniqueness_criteria = ['chain', 'residue_num', 'residue_name']),
                    HierarchyScheme(name='residues',
                                    uniqueness_criteria = ['chain', 'residue_index', 'residue_type'])]
    with pytest.raises(HierarchySchemeAlreadyRegisteredForName, match='...') as context:
        topology = Topology(hierarchy_schemes=hier_schemes)
    
```
* replace_molecule
* register_hierarchy_scheme

```python

 # Let's say that a HIER_FLAVOR is just a list of HierarchySchemes    

def test_topology_register_deregister_hierarchy(self):
    """Test a new hierarchy scheme is succesfully registered in the topology."""
    topology = Topology()
    # register new hierarchy scheme
    topology.register_hierarchy_schemes(HIER_FLAVOR_DEFAULT)
    with pytest.raises(HierarchySchemeAlreadyRegisteredForName, match='...') as context:
        topology.register_hierarchy_schemes(HIER_FLAVOR_DEFAULT)
    assert len(HIER_FLAVOR_DEFAULT) == len(topology.hierarchy_schemes)
    
    for hier_scheme in HIER_FLAVOR_DEFAULT:
        assert topology.hierarchy_schemes[hier_scheme.name] == hier_scheme
    
    hier_scheme_name_to_deregister = HIER_FLAVOR_DEFAULT[0].name
    topology.deregister_hierarchy_schemes([hier_scheme_name_to_deregister])
    assert len(HIER_FLAVOR_DEFAULT) - 1 == len(topology.hierarchy_schemes)
    assert hier_scheme_name_to_deregister not in topology.hierarchy_schemes
    with pytest.raises(HierarchySchemeNotFound, match='...') as context:
        topology.deregister_hierarchy_schemes([hier_scheme_name_to_deregister])


@pytest.parametrize('use_from_molecules', (False, True))
def test_topology_hierarchy_transfer_from_top_mols(self, use_from_molecules):
    """Test hierarchy schemes are transferred from the topology molecules"""
    # IP: This may be more suited for the Topology from_molecules method/API point.
    molecule1 = Molecule(hierarchy_schemes=HIER_FLAVOR_DEFAULT)
    molecule2 = Molecule(hierarchy_schemes=HIER_FLAVOR_DEFAULT)
    if use_from_molecules:
        topology = Topology.from_molecules([molecule1, molecule2])
    else:
        topology = Topology()
        topology.add_molecule(molecule1)
        topology.add_molecule(molecule2)
    assert len(topology.hierarchy_schemes) == len(molecule1.hierarchy_schemes)
    for top_hier_name, top_hier_scheme in topology.hierarchy_schemes.items():
        assert top_hier_scheme.uniqueness_criteria == molecule1.hierarchy_schemes[top_hier_name].uniqueness_criteria
        

@pytest.parametrize('use_from_molecules', (False, True))
def test_topology_hierarchy_transfer_from_top_mols_partial_overlap(self, use_from_molecules):
    hier_scheme_1 = HierarchyScheme(name='residues', 
                                       uniqueness_criteria = ['chain', 'residue_num', 'residue_name'])
    hier_scheme_2 = HierarchyScheme(name='chains',
                                       uniqueness_criteria = ['chain'])
    molecule1 = Molecule(hierarchy_schemes=[hier_schemes_1])
    molecule2 = Molecule(hierarchy_schemes=[hier_schemes_1, hier_schemes_2])
    if use_from_molecules:
        topology = Topology.from_molecules([molecule1, molecule2])
    else:
        topology = Topology()
        topology.add_molecule(molecule1)
        topology.add_molecule(molecule2)
    assert len(topology.hierarchy_schemes) == 2
    assert topology.hierarchy_schemes[hier_schemes_mol2.name] == hier_schemes_2

@pytest.parametrize('use_from_molecules', (False, True))
def test_topology_hierarchy_transfer_from_top_mols_no_overlap(self, use_from_molecules):
    hier_schemes_mol1 = HierarchyScheme(name='residues', 
                                       uniqueness_criteria = ['chain', 'residue_num', 'residue_name'])
    hier_schemes_mol2 = HierarchyScheme(name='chains',
                                       uniqueness_criteria = ['chain'])
    molecule1 = Molecule(hierarchy_schemes=[hier_schemes_mol1])
    molecule2 = Molecule(hierarchy_schemes=[hier_schemes_mol2])
    
    if use_from_molecules:
        topology = Topology.from_molecules([molecule1, molecule2])
    else:
        topology = Topology()
        topology.add_molecule(molecule1)
        topology.add_molecule(molecule2)
    assert len(topology.hierarchy_schemes) == 2
    assert topology.hierarchy_schemes[hier_schemes_mol1.name] == hier_schemes_mol1
    assert topology.hierarchy_schemes[hier_schemes_mol2.name] == hier_schemes_mol2

@pytest.parametrize('use_from_molecules', (False, True))
def test_topology_hierarchy_transfer_from_top_mols_incompatible_partial_overlap(self, use_from_molecules):
    hier_schemes_mol1 = [HierarchyScheme(name='residues', 
                                       uniqueness_criteria = ['chain', 'residue_num', 'residue_name']),
                         HierarchyScheme(name='chains',
                                       uniqueness_criteria = ['chain'])
                        ]
    hier_schemes_mol2 = [HierarchyScheme(name='residues',
                                       uniqueness_criteria = ['chain', 'residue_index', 'residue_type']),
                         HierarchyScheme(name='chains',
                                       uniqueness_criteria = ['chain'])
                        ]
    molecule1 = Molecule(hierarchy_schemes=hier_schemes_mol1)
    molecule2 = Molecule(hierarchy_schemes=hier_schemes_mol2)
    
    with pytest.raises(IncompatibleHierarchySchemeError, match=f'Cannot combine hierarchy schemes with name "{hier_scheme_mol1.name}"'):
        if use_from_molecules:
            topology = Topology.from_molecules([molecule1, molecule2])
        else:
            topology = Topology()
            topology.add_molecule(molecule1)
            topology.add_molecule(molecule2)

@pytest.parametrize('use_from_molecules', (False, True))
def test_topology_hierarchy_transfer_from_top_mols_incompatible_overlap_uniqueness_order(self, use_from_molecules):
    hier_schemes_mol1 = [HierarchyScheme(name='residues', 
                                       uniqueness_criteria = ['chain', 'residue_num', 'residue_name'])
                        ]
    hier_schemes_mol2 = [HierarchyScheme(name='residues',
                                       uniqueness_criteria = ['residue_num', 'residue_name', 'chain'])
                        ]
    molecule1 = Molecule(hierarchy_schemes=hier_schemes_mol1)
    molecule2 = Molecule(hierarchy_schemes=hier_schemes_mol2)
    
    with pytest.raises(IncompatibleHierarchySchemeError, match=f'Cannot combine hierarchy schemes with name "{hier_scheme_mol1.name}"'):
        if use_from_molecules:
            topology = Topology.from_molecules([molecule1, molecule2])
        else:
            topology = Topology()
            topology.add_molecule(molecule1)
            topology.add_molecule(molecule2)
```

* deregister_hierarchy_scheme
    * Already tested in register_hierarchy_scheme
* mdtraj_select
```python
def test_topology_mdtraj_selection(self):
    """Test selection gives the same atoms compared to mdtraj topology selection."""
    molecule1 = Molecule.from_pdb_file(...)
    molecule2 = Molecule.from_pdb_file(...)
    topology = Topology.from_molecules([molecule1, molecule2])
    selection_str = 'name CA and resid 7 to 10'
    mdtraj_top = topology.to_mdtraj()
    mdtraj_output = mdtraj_top.select(selection_str)
    # IP: Do we expect orders to be the same?
    assert topology.mdtraj_select(selection_str) == mdtraj_output
```
* perceive_residues
```python
def test_topology_perceive_residues_iterator(self):
    """Test residue iterator perception."""
    protein = Molecule.from_file(...)
    topology = Topology()
    topology.add_molecule(protein)
    # Perceive residues at both molecule and top levels
    protein.perceive_residues()
    topology.perceive_residues()
    
    assert topology.residues == protein.residues
```
* perceive_hierarchy
* residues (and other iterators) 
* add_molecule (update to include copying in metadata, TypedMolecules, appending hierarchies)

```python
def test_topology_add_single_molecule_copy_metadata(self):
    """Test metadata is copied from atoms to topology atoms using a single molecule."""
    molecule = Molecule.from_file(...)
    # IP -- Would this fill the metadata?
    molecule.perceive_hierarchy(...)
    topology = Topology()
    toplogy.add_molecule(molecule)
    # loop through all atoms and check metadata
    for top_atom, mol_atom in zip(topology.topology_atoms, molecule1.atoms):
        assert top_atom.metadata == mol_atom.metadata

def test_topology_add_typed_molecule(self):
    """Test TypedMolecule is correctly added to the topology molecules."""
    typed_molecule = TypedMolecule.from_file(...)
    topology= Topology()
    topology.add_molecule(typed_molecule)
    assert len(topology.topology_molecules) == 1
    assert isinstance(topology.reference_molecules[0], TypedMolecule)

topology.topology_molecules[0].residue[('A', 10, 'ALA')] --> HierElement or KeyError
topology.residues[('A',10,'ALA')] --> list/iterator of HierElement??

class Topology:
    def residues(self):
        for top_mol in self.topology_molecules:
            if hasattr('residues', top_mol):
                for residue in top_mol.residues:
                    yield residue
    
    # Looks like IP's "get_residues"
    def residue(self, item): # item is tuple, or integer index
        # result = []
        for top_mol in self.topology_molecules:
            if hasattr('residues', top_mol):
                try:
                    yield top_mol.residue(item)
                    # result.append(top_mol.residue(item)
                except KeyError:
                    pass
        # return result
        

                
    
def test_topology_add_molecule_hier_element_collision(self):
    """Test that overlapping topology residues get extended when adding molecules."""
    # This could be useful for testing you get duplicated elements from the same query
    protein1 = Molecule.from_pdb_file(...)
    n_residues_protein1 =  len(protein1.residues)
    topology = Topology()
    topology.add_molecule(protein1)
    topology.add_molecule(protein1)
    assert len(topology.residues) == 2*len(protein1.residues)
    ### 2021-04-29 --Let's leave the expected behavior undefined for now -- Will depend
    # Check first part is NOT still protein1.residues, since each atom should be present twice
    ala_10s_found = 0
    for residue in topology.residues:
        if residue.identifier == ('A', 10, 'ALA'):
            ala_10s_found += 1
    assert ala_10s_found == 2
    assert len(topology.residues[('A', 10, 'ALA')]) == 2*len(protein1.residues[('A', 10, 'ALA')])
    
    
    #    assert topology.residues[residue_idx].topology_atom_indices == protein1.residues[residue_idx].atom_indices
    
    
def test_topology_add_molecule_extend_residues(self):
    """Test topology residues get extended when adding molecules."""
    protein1 = Molecule.from_pdb_file(...)
    protein2 = Molecule.from_pdb_file(...)
    n_residues_protein1 =  len(protein1.residues)
    n_residues_protein2 =  len(protein2.residues)
    total_residues = n_residues_protein1 + n_residues_protein2
    topology = Topology()
    topology.add_molecule(protein1)
    for residue_idx in range(n_residues_protein1):
        assert topology.residues[residue_idx].topology_atom_indices == protein1.residues[residue_idx].atom_indices
    topology.add_molecule(protein2)
    # Check the rest corresponds to protein2.residues
    # TODO: Check indices , make them work!
    for residue_idx in range(n_residues_protein1, total_residues):
        assert topology.residues[residue_idx].topology_atom_indices == protein1.residues[residue_idx].atom_indices
    
def test_topology_add_molecule_extend_iterators(self):
    """Test that all common compatible iterators from the molecules are included in the topology"""
    protein1 = Molecule.from_pdb_file(...)
    protein2 = Molecule.from_pdb_file(...)
    # Assume the molecules already have some exposed iterators and hierarchy schemes
    all_hierarchy_schemes = copy.deepcopy(protein.hierarchy_schemes)
    all_hierarchy_schemes.extend(protein2.hierarchy_schemes)
    all_hierarchy_names = []
    ### STUCK    
```
* to_openmm (update)
* to_mdtraj (update)



### TopologyAtom

* metadata -- API point. Data lives in the topology molecule.

```python
def test_topology_atom_metadata_from_molecule(self):
    
```

### TopologyMolecule

* `__init__()` 
    * update with atom_metadata and make sure ref_mol does not have it
    
```python
def test_topology_molecule_fill_atom_metadata(self):
    """Test topology molecule metadata gets atom metadata from molecule."""
    molecule = Molecule.frome_file(...)
    # This fills metadata
    molecule.perceive_residues()
    topology = Topology()
    topology_molecule = TopologyMolecule(reference_molecule=molecule,
                                         topology=topology
                                        )
    assert topology_molecule.metadata == some_molecule.metadata
    
def test_topology_molecule_reference_molecules_no_metadata(self):
    """Test reference molecules have empty metadata"""
    molecule = Molecule.from_file(...)
    molecule.perceive_residues()
    topology = Topology()
    topology_molecule = TopologyMolecule(reference_molecule=molecule,
                                         topology=topology
                                        )
    reference_molecule = topology.topology_molecules[0].reference_molecule
    assert reference_molecule.metadata is None
    # OR
    assert reference_molecule.metadata == {}
```
    * Take first conformer (if present) as coordinates

```python
def test_topology_molecule_take_first_conformer(self):
    """Test topology molecule gets first conformer from molecule."""
    molecule = Molecule.from_pdb_file(...)
    topology = Topology()
    # Should be just add_molecule?
    topology_molecule = TopologyMolecule(reference_molecule=molecule,
                                         topology=topology
                                        )
    # Currently TopologyMolecule doesn't have conformers
    assert topology_molecule.n_conformers == 1
    assert topology_molecule.conformers[0] == molecule.conformers[0] 
```
* perceive_residues
* perceive_hierarchy
* residues (and other iterators) 
* ref_molecule (Test it is a copy of the reference Molecule)

```python
def test_topology_molecule_reference_molecule_is_new_object(self):
    """Test topology reference molecules are new objects and not copies of
    the referenced molecules."""
    some_molecule = Molecule.from_file(...)
    some_topology = Topology()
    some_topology.add_molecule(some_molecule)
    reference_molecule = some_topology.topology_molecules[0].reference_molecule
    assert reference_molecule is not some_molecule
```

* atom_metadata
* coordinates -- let users specify a single correctly shaped array.

```python
def test_topology_molecule_coordinates_wrong_shape(self):
    """Test topology molecules coordinates do not accept wrong shapes."""
    # Create a wrong sized matrix and try using it as coordinates
    molecule = Molecule.from_file(...)
    topology = Topology()
    topology.add_molecule(molecule)
    # Wrong number of rows/atoms
    with pytest.raises(IncorrectCoordinateArrayShape, match='...'):
        coordinates_array = np.random.rand(molecule.n_atoms+1, 3) * unit.angstroms
        topology.topology_molecule[0].coordinates = coordinates_array
    # Wrong number of dimensions
    with pytest.raises(IncorrectCoordinateArrayShape, match='...'):
        coordinates_array = np.random.rand(molecule.n_atoms, 2) * unit.angstroms
        topology.topology_molecule[0].coordinates = coordinates_array
        
def test_topology_molecule_coordinates_right_shape(self):
    """Test coordinates array has the right shape."""
    molecule = Molecule.from_pdb_file(...)
    topology = Topology()
    topology.add_molecule(molecule)
    # Assuming it automatically transfers the coordinates from the molecule
    n_atoms = molecule.n_atoms
    assert np.shape(topology.topology_molecules[0].coordinates) == (n_atoms, 3)
```



### HierarchyFlavor
* 



### HierarchyScheme
* `__init__(uniqueness_key, iterator_name)`
* `__eq__ (for whether two schemes can safely be appended)`
    * JW -- Don't think we need this.
    * What we really mean when two schemes are equal.
* is_compatible_with
* parent
* elements (return list of HierarchyElement's)
* Test uniqueness_criteria order (different order -> different schemes)
* name -- check for safety (registry behavior)
* `__getitem__` (accept: tuple, integer or slice/list)

```python

```

### MoleculeHierarchyElement

* `__init__()`
* `__eq__`
* atoms
* atom_indices
* hierarchy_scheme
* index/identifier
* Nice to have, but not necessary: EITHER
    * (best case) magical indirection that allows `protein.chains[0].residues[10]`
    * magical slicing that allows `protein.chains[0].iterators.residues[10]`
    * magical slicing that allows `protein.chains[0].iterators['residues'][10]`
    * magical slicing that allows `protein.iterators.chains[0].iterators['residues'][10]`

* iterators (returns self.hierarchy_scheme.topology.hierarchy_schemes)

```python
protein.iterators['chains'][1].iterators['residues'][10]

for chain in protein.chains:
    for residue in chain.residues:
        ...

class TopologyHierarchyElement:
    def iterators(self):
        new_iterators = {}
        sel = set(self.topology_atom_indices)
        for hierarchy_name, hierarchy_scheme in self.hierarchy_scheme.topology.hierarchy_schemes.items():
            new_hier_scheme =  copy.deepcopy(hierarchy_scheme)
            new_hier_scheme.elements = []
            for element in hierarchy_scheme.elements:
                element_atom_indices = set(element.topology_atom_indices)
                if len(sel.intersection(element_atom_indices)):
                    new_hier_scheme.elements.append(copy.deepcopy(element))
            new_iterators[hierarchy_name] = new_hier_scheme
        return new_iterators
                        

```

### TopologyHierarchyElement

* `__init__()`
* `__eq__`
* topology_atoms
* topology_atom_indices
* topology_molecule_atom_indices
* hierarchy_scheme
* index/identifier


* Test running merge_molecules on two molecules that have an existing hierarchies with the SAME name, but DIFFERENT HierarchySchemes behind them. 
    * An error should be raised, instructing the user to EITHER run merge_molecules with transfer_hierarchy_schemes=False
      OR delete one of the colliding hierarchy schemes before merging.
    * Upon deleting the colliding HierarchyScheme from the second molecule, the iterator should immediately dissapear
      (`mol2.delete_hier_scheme('residues')` `mol2.residues --> AttributeError`) 
    * If both mol1 and mol2 have iterators with the same name, they should be appended
    * If ONLY mol1 OR mol2 have an iterator, the final mol should have that iterator, but immediately following
      the merge, only some of the atoms in the new molecule (only those originally in iterators) should be included
    * Atoms that were deleted should not appear in any iterators after the merge
    * Residues/HierElements that had all their atoms deleted MAY appear in the new molecule
    * Residues/HierElements with the same value of `uniqueness_key` should be merged in the new molecule

* Test that running perceive_residues on (a TypedMolecule, or a TopologyMolecule with its reference as a typed molecule, or a Topology containing a TypedMolecule) does NOT clear the existing data 

```python
new_mol.perceive_hierarchy([atom_indices], [hierarchy_scheme_names])
#new_mol.perceive_hierarchy({'residues': Molecule.HierScheme, 'chains': Molecule2.HierScheme})
new_mol.register_hierarchy_scheme(hier_scheme_with_name_residues)
new_mol.register_hierarchy_scheme(hier_scheme_with_name_chains)
new_mol.perceive_hierarchy(['residues', 'chains'])                            
new_mol.hierarchy_schemes
> {'residues': <HierScheme with name 'residues'>, 'chains': <HierScheme2 with name 'chains'>...}
```